# Streaming Consumer Demo for Kinesis and Kafka

In [48]:
from boto import kinesis
import datetime
import time

# Kinesis

In [49]:
STREAM_NAME="latency"
NUMBER_PARTITIONS=1

In [50]:
kinesis_conn = kinesis.connect_to_region("us-east-1")
streams = kinesis_conn.list_streams() 
if STREAM_NAME not in streams["StreamNames"]:
    print "Create new Stream"
    kinesis_conn.create_stream(STREAM_NAME, NUMBER_PARTITIONS)

In [51]:
kinesis_conn.describe_stream(STREAM_NAME)

{u'StreamDescription': {u'EnhancedMonitoring': [{u'ShardLevelMetrics': []}],
  u'HasMoreShards': False,
  u'RetentionPeriodHours': 24,
  u'Shards': [{u'HashKeyRange': {u'EndingHashKey': u'340282366920938463463374607431768211455',
     u'StartingHashKey': u'0'},
    u'SequenceNumberRange': {u'StartingSequenceNumber': u'49569120833560767652834857066126646824474736685823819778'},
    u'ShardId': u'shardId-000000000000'}],
  u'StreamARN': u'arn:aws:kinesis:us-east-1:157926643553:stream/latency',
  u'StreamCreationTimestamp': 1483312335.0,
  u'StreamName': u'latency',
  u'StreamStatus': u'ACTIVE'}}

In [52]:
kinesis_conn.put_record(STREAM_NAME, "hi", str(time.time()))

{u'SequenceNumber': u'49569120833560767652834857071632095006999778700717719554',
 u'ShardId': u'shardId-000000000000'}

In [53]:
def produce_kinesis(payload):
     kinesis_conn.put_record(STREAM_NAME, payload, str(time.time())) # use random partition key based on time

# Benchmark

In [55]:
#sent_time=datetime.datetime.now().isoformat()
sleep_time = 1

for sleep_time in [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]:
#for sleep_time in [1]:
    for i in range(20):
        payload_string = datetime.datetime.now().isoformat() + ";" + str(sleep_time)
        produce_kinesis(payload_string)
        print "sent @ " + datetime.datetime.now().isoformat()
        time.sleep(sleep_time)

sent @ 2017-01-01T17:38:07.390633
sent @ 2017-01-01T17:38:08.447301
sent @ 2017-01-01T17:38:09.503785
sent @ 2017-01-01T17:38:10.560005
sent @ 2017-01-01T17:38:11.697737
sent @ 2017-01-01T17:38:12.754380
sent @ 2017-01-01T17:38:13.810557
sent @ 2017-01-01T17:38:14.866777
sent @ 2017-01-01T17:38:15.926039
sent @ 2017-01-01T17:38:16.983161
sent @ 2017-01-01T17:38:18.039869
sent @ 2017-01-01T17:38:19.096336
sent @ 2017-01-01T17:38:20.152714
sent @ 2017-01-01T17:38:21.209376
sent @ 2017-01-01T17:38:22.265262
sent @ 2017-01-01T17:38:23.321966
sent @ 2017-01-01T17:38:24.381472
sent @ 2017-01-01T17:38:25.438651
sent @ 2017-01-01T17:38:26.495265
sent @ 2017-01-01T17:38:27.554905
sent @ 2017-01-01T17:38:28.615639
sent @ 2017-01-01T17:38:28.771506
sent @ 2017-01-01T17:38:28.926386
sent @ 2017-01-01T17:38:29.082562
sent @ 2017-01-01T17:38:29.280809
sent @ 2017-01-01T17:38:29.436503
sent @ 2017-01-01T17:38:29.591204
sent @ 2017-01-01T17:38:29.746900
sent @ 2017-01-01T17:38:29.902388
sent @ 2017-01

# Cleanup

In [56]:
kinesis_conn.delete_stream(STREAM_NAME)